# ERA5 yearly datasets 

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# Paths to my data directories
cwd_path = Path.cwd()
data_path = cwd_path.parent.joinpath('data')
data_push_path = cwd_path.parent.joinpath('data_to_push')

In [26]:
# Print files in my ERA5 dir
for filepath in data_path.joinpath('ERA5').iterdir():
    print(filepath)

/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2020.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2023_old.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2021.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_combined.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2018.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2019.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2023_the_whole_year_how.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2022.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2023.nc


#### Note:
- Note: year 2023 has an unexpected dim called `expver`
    - "expver is used to tell the difference between the initial release (expver=5, called ERA5T) and validated ERA5 data (expver=1). In most cases, ERA5 is identical to ERA5T"
    - More here: https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#ERA5:datadocumentation-Dataupdatefrequency

### Read ERA5 datasets into Xarray

In [7]:
ds_2018 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2018.nc'))

In [10]:
ds_2019 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2019.nc'))

In [11]:
ds_2020 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2020.nc'))

In [12]:
ds_2021 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2021.nc'))

In [13]:
ds_2022 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2022.nc'))

In [27]:
ds_2023 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2023.nc'))

#### Concat the xarray datasets along dimension `time`

In [29]:
datasets = [ds_2018, ds_2019, ds_2020, ds_2021, ds_2022, ds_2023]
ds_combined = xr.concat(datasets, dim='time')

In [31]:
# ds_combined.time.to_dataframe()['time'].value_counts()

#### Save combined dataset to file and check size
- Should I go ahead and subset against used grid points?
- Get latest used grid points from latest turbine df

In [32]:
ds_combined.to_netcdf(data_path.joinpath('ERA5').joinpath('era5_combined.nc'))

In [33]:
# Load back into xarray and check structure
ds = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_combined.nc'))

In [35]:
ds_combined

<xarray.Dataset>
Dimensions:    (longitude: 52, latitude: 42, time: 48168)
Coordinates:
  * longitude  (longitude) float32 4.22 4.47 4.72 4.97 ... 16.47 16.72 16.97
  * latitude   (latitude) float32 56.32 56.07 55.82 55.57 ... 46.57 46.32 46.07
  * time       (time) datetime64[ns] 2018-01-01 ... 2023-06-30T23:00:00
Data variables:
    u100       (time, latitude, longitude) float32 9.292 9.183 ... 4.81 4.561
    v100       (time, latitude, longitude) float32 7.208 7.531 ... 2.707 2.714
    u10        (time, latitude, longitude) float32 7.189 7.147 ... 1.702 1.548
    v10        (time, latitude, longitude) float32 5.964 6.266 ... 1.817 1.81
    t2m        (time, latitude, longitude) float32 281.0 281.0 ... 293.0 292.8
    i10fg      (time, latitude, longitude) float32 12.9 13.21 ... 5.885 5.432
    msl        (time, latitude, longitude) float32 9.838e+04 ... 1.009e+05
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-11 11:36:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [34]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 52, latitude: 42, time: 48168)
Coordinates:
  * longitude  (longitude) float32 4.22 4.47 4.72 4.97 ... 16.47 16.72 16.97
  * latitude   (latitude) float32 56.32 56.07 55.82 55.57 ... 46.57 46.32 46.07
  * time       (time) datetime64[ns] 2018-01-01 ... 2023-06-30T23:00:00
Data variables:
    u100       (time, latitude, longitude) float32 ...
    v100       (time, latitude, longitude) float32 ...
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    i10fg      (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-11 11:36:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

-----

## Explore 

In [8]:
ds_2018

<xarray.Dataset>
Dimensions:    (longitude: 52, latitude: 42, time: 8760)
Coordinates:
  * longitude  (longitude) float32 4.22 4.47 4.72 4.97 ... 16.47 16.72 16.97
  * latitude   (latitude) float32 56.32 56.07 55.82 55.57 ... 46.57 46.32 46.07
  * time       (time) datetime64[ns] 2018-01-01 ... 2018-12-31T23:00:00
Data variables:
    u100       (time, latitude, longitude) float32 ...
    v100       (time, latitude, longitude) float32 ...
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    i10fg      (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-11 11:36:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [6]:
ds_2023

<xarray.Dataset>
Dimensions:    (longitude: 52, latitude: 42, expver: 2, time: 5832)
Coordinates:
  * longitude  (longitude) float32 4.22 4.47 4.72 4.97 ... 16.47 16.72 16.97
  * latitude   (latitude) float32 56.32 56.07 55.82 55.57 ... 46.57 46.32 46.07
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 2023-01-01 ... 2023-08-31T23:00:00
Data variables:
    u100       (time, expver, latitude, longitude) float32 ...
    v100       (time, expver, latitude, longitude) float32 ...
    u10        (time, expver, latitude, longitude) float32 ...
    v10        (time, expver, latitude, longitude) float32 ...
    t2m        (time, expver, latitude, longitude) float32 ...
    i10fg      (time, expver, latitude, longitude) float32 ...
    msl        (time, expver, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-11 18:05:31 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

-----

### Check that all lats and lons match between data sets!
- Add 2023 dataset to check and run again!

In [24]:
def check_dimensions_match(datasets, dim_name):
    """Check if a given dimension's values are the same across multiple datasets."""
    reference_values = datasets[0][dim_name].values
    for ds in datasets[1:]:
        if not (ds[dim_name].values == reference_values).all():
            return False
    return True

In [25]:
# Check latitude and longitude for all datasets
latitude_match = check_dimensions_match(datasets, 'latitude')
longitude_match = check_dimensions_match(datasets, 'longitude')

if latitude_match and longitude_match:
    print("All datasets have matching latitude and longitude values!")
else:
    if not latitude_match:
        print("Datasets do not have matching latitude values.")
    if not longitude_match:
        print("Datasets do not have matching longitude values.")

All datasets have matching latitude and longitude values!


------

## Derive 10m and 100m mean wind speeds 
- Subset the data first?!

In [ ]:
def calc_wind_speed_using_ortho_components(u_zonal, v_meridional):
    """
    u_zondal (u10): wind component along local parallel of latitude; positive from west, negative from east
    v_meridional (v10): wind component along local meridian; positive from south, negative from north
    returns the magnitude of the wind vector (i.e. wind speed)
    """
    # use numpy.sqrt() as math.sqrt() only accepts scalar value not array
    mean_wind_speed = np.sqrt(u_zonal**2 + v_meridional**2)
    return mean_wind_speed